In [244]:
%pip install particle
%pip install sympy

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


Lets start by defining a decay. In the code beow you can see how to define a simple utility, which given momenta will be able to claculate all needed rotations.

For simplicity we will choose a simple
As you see, the isobars are allways isolated, meaning no isobar is the predecessor of another isobar. This makes the the full amplitude straightforward to calculate.

In [245]:
from decayangle.decay_topology import TopologyGroup, Node
import numpy as np

# Find the possible the decay topologies
tg = TopologyGroup(0, [1,2,3])
for decay_tree in tg.trees:
    print(decay_tree)

( 0 -> ( (2, 3) -> 2, 3 ), 1 )
( 0 -> ( (1, 3) -> 1, 3 ), 2 )
( 0 -> ( (1, 2) -> 1, 2 ), 3 )


First lets define some basic utility functions. This is not something you need to understand 100% on the first try.
What we do is set up the basic functional setup to be able to perform a three body analysis, which is the simplest case. 

In [246]:
from typing import NamedTuple, Callable
from math import prod
from particle import Particle


def wigner_small_d(theta, j, m1, m2):
    """Calculate Wigner small-d function. Needs sympy.
      theta : angle
      j : spin (in units of 1/2, e.g. 1 for spin=1/2)
      m1 and m2 : spin projections (in units of 1/2)

    :param theta:
    :param j:
    :param m1: before rotation
    :param m2: after rotation

    """
    from sympy import Rational
    from sympy.abc import x
    from sympy.utilities.lambdify import lambdify
    from sympy.physics.quantum.spin import Rotation as Wigner
    j,m1,m2 = int(j),int(m1),int(m2)
    # TODO: check if this is correct (the order of the m1 and m2)
    d = Wigner.d(Rational(j, 2), Rational(m2, 2), Rational(m1, 2), x).doit().evalf()
    d = lambdify(x, d, "jax")(theta)
    return d

    
def BWResonance(spin, mass, width):
    """Create a Breit-Wigner resonance function for a given spin.
    Args:
        spin (int): spin quantum number multiplied by 2
    """
    def f(s, L):
        return np.sqrt(s) / (s - mass**2 + 1j * mass * width)
    return f

class resonance:
    def __init__(self, spin, parity, mass, width, name, mother:Particle, bachelor:Particle, isobar: tuple):
        self.spin = spin
        self.mass = mass
        self.width = width
        self.lineshape = BWResonance(spin, mass, width)
        self.parity = parity
        self.name = name
        self.mother = mother
        self.bachelor = bachelor
        self.isobar = isobar


    @property
    def possible_helicities(self):
        return list(range(-self.spin, self.spin+1, 2))
    
    @property
    def isbar_spins(self):
        return [p.spin for p in self.isobar]
    
    @property
    def isobar_parities(self):
        return [p.parity for p in self.isobar]

    def LS_couplings_mother_decay(self):
        allowed_ls = []
        LSCoupling = NamedTuple('LSCoupling', [('L', int), ('S', int), ('coupling', complex)])
        for L in range(0, self.mother.spin + self.spin + self.bachelor.spin + 6, 2):
            for S in range(abs(L - self.mother.spin), L + self.mother.spin + 1, 2):
                if abs(L - S) <= self.mother.spin <= L + S:
                    allowed_ls.append((L, S))
        return {
            f"{self.name}:motherDecay:L{L}.S{S}": LSCoupling(L, S, 1 + 0j) for L, S in allowed_ls
        }
    
    def LS_coupling_resonance_decay(self):
        allowed_ls = []
        LSCoupling = NamedTuple('LSCoupling', [('L', int), ('S', int), ('coupling', complex)])
        for L in range(0, self.spin + sum(self.isbar_spins) + self.mother.spin, 2):
            for S in range(abs(L - self.spin), L + self.spin + 1, 2):
                if abs(L - S) <= self.spin <= L + S: # check J double
                    if prod(self.isobar_parities) * (-1)**(L) == self.parity: # check parity
                        allowed_ls.append((L, S))
        return {
            f"{self.name}:resonanceDecay:L{L}.S{S}": LSCoupling(L, S, 1 + 0j) for L, S in allowed_ls
        }
            
    def clebsch_gordan(self, j1, m1, j2, m2, J, M):
        """
        Return clebsch-Gordan coefficient. Note that all arguments should be multiplied by 2
        (e.g. 1 for spin 1/2, 2 for spin 1 etc.). Needs sympy.
        """
        from sympy.physics.quantum.cg import CG
        from sympy import Rational
        cg = CG(
                Rational(j1, 2),
                Rational(m1, 2),
                Rational(j2, 2),
                Rational(m2, 2),
                Rational(J, 2),
                Rational(M, 2),
            ).doit().evalf()
        return float(cg)
    
    def helicity_coupling_times_lineshape(self, s, hi_, hj_):
        ls_resonance_decay = self.LS_coupling_resonance_decay()
        pi, pj = self.isobar
        h = sum(
            ls.coupling * self.lineshape(s, ls.L) * self.clebsch_gordan(pi.spin, -hi_, pj.spin, hj_, ls.S, hj_ - hi_) * self.clebsch_gordan(ls.S, hj_ - hi_, ls.L, 0 , self.spin, hi_ - hj_)
            for ls in ls_resonance_decay.values()
        )
        return h

    
    
    def h_mother(self,hk_, hiso_) -> float:
        # TODO: calculate the h values from the LS values and  clebsch gordans
        mother_decay = self.LS_couplings_mother_decay()
        pk = self.bachelor
        return sum(
            ls.coupling *  self.clebsch_gordan(self.spin, hiso_, pk.spin, -hk_, ls.S, hiso_ - hk_) * self.clebsch_gordan(ls.S, hiso_ - hk_, ls.L, 0 , self.mother.spin, hiso_ - hk_)
            for ls in mother_decay.values()
        )
       

        

possible_helicities = lambda spin: list(range(-spin, spin+1, 2))

The particle porperites can be fetched directly from the PDG, with the help of the particles project.

We go for a decay of $\Lambda_b \rightarrow \Lambda_c^+ D^0 K^-$

In [247]:
# shortcut to get the pdg parameters for our particles
class particle:
    def __init__(self, p):
        self.p = p 
    
    @property
    def spin(self):
        return int(self.p.J * 2)

    @property
    def parity(self):
        return self.p.P

    def __getattr__(self, attr):
        return getattr(self.p, attr)
    
    def helicities(self):
        return possible_helicities(self.spin)

p0 = particle(Particle.findall('Lambda(b)0')[0])
p1 = particle(Particle.findall('D0')[0])
p2 = particle(Particle.findall('Lambda(c)+')[0])
p3 = particle(Particle.findall('K-')[0])

particles = {
    0: p0,
    1: p1,
    2: p2,
    3: p3
}

for i in range(4):
    print(f"Particle {i}: {particles[i].name} spin: {particles[i].spin} helicities: {particles[i].helicities()}")


Particle 0: Lambda(b)0 spin: 1 helicities: [-1, 1]
Particle 1: D0 spin: 0 helicities: [0]
Particle 2: Lambda(c)+ spin: 1 helicities: [-1, 1]
Particle 3: K- spin: 0 helicities: [0]


Now we have the main components for the amplitude. We can describe the lineshape for as single resonance, we can compute helicity couplings and we can calculate all the angles. Only thing left to do is write the loops for the sums. The final form should look somewhat like this:

$\sum_{\mathrm{trees}}\sum_{\{\lambda\}}\sum_{\{\lambda^{'}\}} R(m,\{\lambda^{'}\}) d(\theta, \lambda_k^{'}, \lambda_{ij}^{'}) d(\phi_1,\lambda^{'}_1,\lambda_1)d(\phi_2,\lambda^{'}_2,\lambda_2)d(\phi_3,\lambda^{'}_3,\lambda_3)$

This simply describes the sum over all different possible decay trees, and the sum over all possible helicities.
A second sum over the primes helicities $\{\lambda^{'}\}$ in conjuction with the wigner roatation $d(\theta, \lambda_k^{'}, \lambda_{ij}^{'})$ describes the needed basis change after the rotation to move the $(ij)$ isobar onto the z axis.

The rotations $d(\phi_1,\lambda^{'}_1,\lambda_1)d(\phi_2,\lambda^{'}_2,\lambda_2)d(\phi_3,\lambda^{'}_3,\lambda_3)$ describe the rotations rotating the final states of the different chains to a common constellation.

The $R(m,\{\lambda^{'}\})$ is the lineshape of the resonance together with the couplings $H_{0 \rightarrow (ij)\, k}^{\lambda_0, \lambda_{(ij) - \lambda_k}} \cdot X(m,L) \cdot H_{(ij) \rightarrow i\, j}^{\lambda_{(ij)}, \lambda_i^{'} - \lambda_j^{'}}$. 
The couplings $H$ are alled helicity couplings 

In [248]:
momenta = { 
    1: np.array([0, 0, -0.9, 1]),
    2: np.array([0, 0.15, 0.4,1]),
    3: np.array([ 0, 0.3, 0.3,1]),
}

reference_tree = tg.trees[0]
momenta = reference_tree.to_rest_frame(momenta)

# We can define resonances based on the isobar they can appear in 
resonance_lineshapes = {
    (1, 3): [ 
                resonance(0, 1, 2800, 100, 'D2800', p0, p2, (p1, p3)) 
            ], 
    }

print(resonance(2, 1, 2800, 100, 'D2800', p0, p2, (p1, p3)).LS_coupling_resonance_decay())

{'D2800:resonanceDecay:L0.S2': LSCoupling(L=0, S=2, coupling=(1+0j)), 'D2800:resonanceDecay:L2.S0': LSCoupling(L=2, S=0, coupling=(1+0j)), 'D2800:resonanceDecay:L2.S2': LSCoupling(L=2, S=2, coupling=(1+0j)), 'D2800:resonanceDecay:L2.S4': LSCoupling(L=2, S=4, coupling=(1+0j))}


Now we need to get the values for our rotations. For this we can simply iterate through all the possible decay trees, the ```TopologyGroup``` Object generates for us.

In [249]:
for tree in tg.trees:
        final_state_rotations = {
            target:reference_tree.relative_wigner_angles(tree, target, momenta)
            for target in [1, 2, 3]
        }
        isobars = tree.helicity_angles(momenta)
        print(tree, isobars)

( 0 -> ( (2, 3) -> 2, 3 ), 1 ) {((2, 3), 1): (3.141592653589793, -1.1917396523105919)}
( 0 -> ( (1, 3) -> 1, 3 ), 2 ) {((1, 3), 2): (5.951828758370096e-17, -0.23908244819164334)}
( 0 -> ( (1, 2) -> 1, 2 ), 3 ) {((1, 2), 3): (5.041182631349134e-17, -0.26891484091188184)}


For each tree, we see, that we get exactly one isobar constellation. We also see, that of the two angles we compute, only one is non 0 (or $\pi$).

This is expected, as we are already in the decay plane. Thus this angle should go to 0. Computing it could be left out.

In [250]:
# particle 1 
spin1 = p1.spin
helicities1 = possible_helicities(spin1)

# particle 2
spin2 = p2.spin
helicities2 = possible_helicities(spin2)

# particle 3
spin3 = p3.spin
helicities3 = possible_helicities(spin3)

spin0 = p0.spin
helicities0 = possible_helicities(spin0)

# we can use the tree objects to controll out sums. 
# This makes the function definition straightforward
def f(h0, h1, h2, h3):
    helicity_list = [h0, h1, h2, h3]
    spin_list = [spin0, spin1, spin2, spin3]
    for tree in tg.trees:
        final_state_rotations = {
            target:reference_tree.relative_wigner_angles(tree, target, momenta)[1]
            for target in [1, 2, 3]
        }
        isobars = tree.helicity_angles(momenta)
        for (isobar, bachelor), theta in isobars.items():
            # determinethe correct helicities
            (i,j), k = isobar, bachelor
            hi ,hj, hk = helicity_list[i], helicity_list[j], helicity_list[k]
            si, sj, sk = spin_list[i], spin_list[j], spin_list[k]

            amplitude = sum(
                resonance.helicity_coupling_times_lineshape(tree.nodes[isobar].mass(momenta)**2,hi_, hj_) * wigner_small_d(theta, spin0, h0,  h_iso - hk_) * wigner_small_d(final_state_rotations[i], si, hi, hi_) * wigner_small_d(final_state_rotations[j], sj, hj, hj_) * wigner_small_d(final_state_rotations[k], sk, hk, hk_) * resonance.h_mother(hk_, h_iso)
                for resonance in resonance_lineshapes.get(isobar, [])
                for h_iso in resonance.possible_helicities
                for hk_ in particles[bachelor].helicities()
                for hi_ in particles[isobar[0]].helicities()
                for hj_ in particles[isobar[1]].helicities()
            )
            print(amplitude)

f(1, 0, 1, 0)
        

0
(1.0655444e-08+3.8055173e-10j)
0
